In [1]:
import os
import openai
import json
import random
import copy
import torch
import utils
openai.api_base=''
openai.api_key = ''    #这个用closeai的接口，不需要翻墙
random.seed(10)

In [3]:
#按照空行切割一下原始list，得到每个句子的8个最相似样本
def deal(list_ori,p):   
    list_new=[]				#处理后的列表，是一个二维列表
    list_short=[]			#用于存放每一段列表
    for i in list_ori:
        if i!=p:		
            list_short.append(i)
        else:
            list_new.append(list_short)
            list_short=[]
    #list_new.append(list_short)   #最后一段遇不到切割标识，需要手动放入
    return list_new
#测试成功

In [4]:
def deal_dev(dev_data):
    new_data=[]
    for data in dev_data:
        if 'Triples' in data:
            temp_data=data.replace('Triples','You are expected to output')
            if temp_data[-2]==';':
                    temp_data=temp_data[:-2]+'\n'
            new_data.append(temp_data)
        else:
            new_data.append(data)
    return new_data

In [4]:
def context_gpt(sent_data,context_data,context_num):
    result=[]
    task1='Extract the triples which consist of subject, predicate and object from the sentence. \
Output the triples as <subject|predicate|object> and use ; to split the triples. Following are some examples. '
    for i in range(len(sent_data)):
        prompt=task1+''.join(context_data[i][:2*context_num])
        message=[{"role": "system", "content": prompt}]
        message.append({"role": "user", "content": sent_data[i][:-1]})
        temp_result=utils.gptresult(message)
        result.append(temp_result)
        print('演示数：',context_num,'句子序号：',i)
    return result
#测试通过

#随机选取样本的操作函数
def context_gpt_random(sent_data,tsv_data,context_num):
    result=[]
    task1='Extract the triples which consist of subject, predicate and object from the sentence. \
Output the triples as <subject|predicate|object> and use ; to split the triples. Following are some examples. ' 
    for i in range(len(sent_data)):
        context=''
        random_numbers = random.sample(range(0,int(len(tsv_data)/2),2), context_num)     #取偶数，这样才能得到句子和sample,而且每次都随机取
        for num in random_numbers:
            context+=''.join(tsv_data[num:num+2])
        prompt=task1+context
        message=[{"role": "system", "content": prompt}]
        message.append({"role": "user", "content": sent_data[i][:-1]})
        temp_result=utils.gptresult(message)
        result.append(temp_result)
        print('演示数：',context_num,'句子序号：',i)
    return result
#测试通过

In [5]:
def write_result(result,file_name,sent_list):
    with open(file_name,'w') as f:
        for i in range(len(sent_list)):
            print(i)
            for triple in result[i]:
                if 'Triples:' in triple:
                    triple.replace('Triples:','')
                if '<Triples:' in triple:
                    triple.replace('<Triples:','')
                if triple=='':
                    continue
                triple1=triple.replace('<','')
                triple2=triple1.replace('>','')
                temp_data=triple2.split('|')
                f.write(sent_list[i][:-1]+'\t <arg1> '+temp_data[0]+' </arg1> <rel> '+temp_data[1]+' </rel> <arg2> '+temp_data[2]+' </arg2>\t0.1\n')

In [6]:
with open('../data/carb/carb_50.txt','r') as f:     
    sent_50=f.readlines()
with open('../data/carb/carb_context_byBERTandConsti.txt','r') as f:
    dev_data=f.readlines()
new_dev=deal_dev(dev_data)
context_data=deal(new_dev,'\n')

In [ ]:
sent_data=sent_50
for context_num in range(3,9):
    result=[]
    task1='Extract the triples which consist of subject, predicate and object from the sentence. \
Output the triples as <subject|predicate|object> and use ; to split the triples. Following are some examples. '
    for i in range(len(sent_data)):
        prompt=task1+''.join(context_data[i][:2*context_num])
        message=[{"role": "system", "content": prompt}]
        message.append({"role": "user", "content": sent_data[i][:-1]})
        temp_result=utils.gptresult(message)
        result.append(temp_result)
        print('演示数：',context_num,'句子序号：',i)
    write_result(result,'../result/few-shot/CaRB/bertandconsti'+str(context_num)+'.carb',sent_50)